In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/dataset.csv")
df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2024-04-30 23:55:50,2024-04-30 23:59:00,2024-05-01 00:00:36,2024-05-01 00:38:21,138,21,18.68,...,4.48,0.00,2.5,0.00,47.41,N,N,N,N,N
1,HV0003,B03404,B03404,2024-05-01 00:41:32,2024-05-01 00:47:40,2024-05-01 00:49:40,2024-05-01 00:57:08,21,22,1.71,...,1.03,0.00,0.0,0.00,6.69,N,N,N,N,N
2,HV0003,B03404,B03404,2024-05-01 00:09:01,2024-05-01 00:10:51,2024-05-01 00:11:31,2024-05-01 00:33:51,140,129,5.00,...,2.65,0.75,0.0,0.00,19.82,Y,N,N,N,N
3,HV0003,B03404,B03404,2024-05-01 00:25:43,2024-05-01 00:48:00,2024-05-01 00:48:30,2024-05-01 01:10:17,138,48,9.25,...,3.47,2.75,2.5,0.00,25.28,N,N,N,N,N
4,HV0003,B03404,B03404,2024-05-01 00:56:21,2024-05-01 00:59:41,2024-05-01 00:59:45,2024-05-01 01:16:11,164,66,6.36,...,2.72,2.75,0.0,7.39,18.23,N,N,N,N,N


In [3]:
df.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [4]:
df["request_datetime"] = pd.to_datetime(df["request_datetime"])
df["on_scene_datetime"] = pd.to_datetime(df["on_scene_datetime"])
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
df["dropoff_datetime"] = pd.to_datetime(df["dropoff_datetime"])

df['shared_request_flag'] = df['shared_request_flag'].map({'Y': True, 'N': False}).astype('boolean')
df['shared_match_flag'] = df['shared_match_flag'].map({'Y': True, 'N': False}).astype('boolean')
df['access_a_ride_flag'] = df['access_a_ride_flag'].map({'Y': True, 'N': False}).astype('boolean')
df['wav_request_flag'] = df['wav_request_flag'].map({'Y': True, 'N': False}).astype('boolean')
df['wav_match_flag'] = df['wav_match_flag'].map({'Y': True, 'N': False}).astype('boolean')



In [5]:
datetime_dim = df[['request_datetime','on_scene_datetime','dropoff_datetime','pickup_datetime']].drop_duplicates().reset_index(drop=True).copy()
datetime_dim['datetime_id'] = datetime_dim.index
datetime_dim = datetime_dim[['datetime_id','request_datetime','dropoff_datetime','on_scene_datetime','pickup_datetime']]
datetime_dim.head()

,datetime_id,request_datetime,dropoff_datetime,on_scene_datetime,pickup_datetime
0,0,2024-04-30 23:55:50,2024-05-01 00:38:21,2024-04-30 23:59:00,2024-05-01 00:00:36
1,1,2024-05-01 00:41:32,2024-05-01 00:57:08,2024-05-01 00:47:40,2024-05-01 00:49:40
2,2,2024-05-01 00:09:01,2024-05-01 00:33:51,2024-05-01 00:10:51,2024-05-01 00:11:31
3,3,2024-05-01 00:25:43,2024-05-01 01:10:17,2024-05-01 00:48:00,2024-05-01 00:48:30
4,4,2024-05-01 00:56:21,2024-05-01 01:16:11,2024-05-01 00:59:41,2024-05-01 00:59:45


In [6]:
service_mapping = {
    'HV0002': 'Juno',
    'HV0003': 'Uber',
    'HV0004': 'Via',
    'HV0005': 'Lyft'
}

trip_dim = df[['PULocationID','DOLocationID','trip_miles','trip_time','hvfhs_license_num','dispatching_base_num','originating_base_num']].drop_duplicates().reset_index(drop=True).copy()
trip_dim['trip_id'] = trip_dim.index
trip_dim['service_name'] = trip_dim['hvfhs_license_num'].map(service_mapping)
trip_dim = trip_dim[['trip_id','service_name','PULocationID','DOLocationID','trip_miles','trip_time','dispatching_base_num','originating_base_num','hvfhs_license_num']]
trip_dim.head()

,trip_id,service_name,PULocationID,DOLocationID,trip_miles,trip_time,dispatching_base_num,originating_base_num,hvfhs_license_num
0,0,Uber,138,21,18.68,2265,B03404,B03404,HV0003
1,1,Uber,21,22,1.71,448,B03404,B03404,HV0003
2,2,Uber,140,129,5.00,1340,B03404,B03404,HV0003
3,3,Uber,138,48,9.25,1307,B03404,B03404,HV0003
4,4,Uber,164,66,6.36,986,B03404,B03404,HV0003


In [7]:
flags_dim = df[['shared_request_flag','shared_match_flag','access_a_ride_flag','wav_request_flag','wav_match_flag']].drop_duplicates().reset_index(drop=True).copy()
flags_dim['flag_id'] = flags_dim.index
flags_dim = flags_dim[['flag_id','shared_request_flag','shared_match_flag','access_a_ride_flag','wav_request_flag','wav_match_flag']]
flags_dim.head()

,flag_id,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,0,False,False,False,False,False
1,1,True,False,False,False,False
2,2,False,False,False,False,True
3,3,True,True,False,False,False
4,4,True,False,False,False,True


In [9]:
fact_table = df.merge(
    datetime_dim, 
    on=['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime']
).merge(
    trip_dim, 
    on=['PULocationID', 'DOLocationID', 'trip_miles', 'trip_time', 'hvfhs_license_num', 'dispatching_base_num', 'originating_base_num']
).merge(
    flags_dim, 
    on=['shared_request_flag', 'shared_match_flag', 'access_a_ride_flag', 'wav_request_flag', 'wav_match_flag']
)[['datetime_id', 'trip_id', 'flag_id', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay']]

fact_table['vendor_id'] = fact_table.index
new_column_order = [
    'vendor_id',          # Identifier
    'datetime_id',        # Foreign key for time
    'trip_id',            # Foreign key for trip details
    'flag_id',            # Foreign key for flags
    'base_passenger_fare', 
    'tolls', 
    'bcf', 
    'sales_tax', 
    'congestion_surcharge', 
    'airport_fee', 
    'tips', 
    'driver_pay'
]
fact_table = fact_table[new_column_order]

fact_table.head()

,vendor_id,datetime_id,trip_id,flag_id,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay
0,0,0,0,0,48.01,0.00,1.39,4.48,0.00,2.5,0.00,47.41
1,1,1,1,0,11.62,0.00,0.32,1.03,0.00,0.0,0.00,6.69
2,2,3,3,0,29.67,6.94,1.08,3.47,2.75,2.5,0.00,25.28
3,3,4,4,0,30.66,0.00,0.84,2.72,2.75,0.0,7.39,18.23
4,4,5,5,0,20.71,0.00,0.57,1.84,0.00,0.0,0.00,20.11
